# Zugang zu Daten

Es bestehen viele Wege, um Daten für eine Auswertung zu beziehen. Wir wollen daher in diesem Abschnitt auf ein paar gängige Methoden des Datenbezugs eingehen:

* Download und Import kleinerer Dateien wie z.B. csv- oder txt-Dateien
* Verwendung von Datenbanken
* Zugang zu Daten mittels APIs

Dies soll lediglich als kleiner Einblick verstanden werden, um eine Idee davon zu erhalten, welche Wege des Datenzugangs bestehen. Darüber hinaus existieren viele weitere Möglichkeiten. Es gilt jedoch für nahezu jeden dieser Wege, dass er mit steigenden Ansprüchen komplexer werden kann und man für detaillierte Mittel und Wege weiteres Wissen aufbauen muss.

## Dateiimport

Mit die gewöhnlichste Art Zugang zu Daten zu erhalten, ist das Teilen von Dateien. Übliche Formate sind csv oder txt Dateien, bei denen in der Regel jede Zeile einer Beobachtung entspricht und die Variablen durch Komma- oder andere Trennzeichen getrennt sind.

Im der unteren Zelle sehen Sie wie mittels pandas eine csv-Datei eingelesen wird. Sehr ähnlich funktioniert dies auch für txt-Dateien. Je nachdem wie die Daten in der Datei getrennt sind (z.B. Leerzeichen, Komma, etc.), muss dies beim Einlesen der Datei explizit angegben werden. Genauere Hilfe hierzu erhalten Sie durch den *help(pd.read_csv)* Aufruf. Die Datei aus diesem Beispiel (dann mit aktuellen Daten) können Sie selbst relativ leicht unter diesem [Link](https://finance.yahoo.com/quote/DAX/history?p=DAX) durch Klicken auf den Download-Button herunterladen. 

In [1]:
import pandas as pd

dax_df = pd.read_csv("../Daten/DAX.csv")
dax_df

,Date,Open,High,Low,Close,Adj Close,Volume
0,2022-07-19,23.180000,23.719999,23.170000,23.590000,22.978786,74900
1,2022-07-20,23.430000,23.600000,23.280001,23.350000,22.745005,17100
2,2022-07-21,23.150000,23.510000,23.150000,23.510000,22.900860,4900
3,2022-07-22,23.740000,23.740000,23.330000,23.330000,22.725523,4700
4,2022-07-25,23.600000,23.600000,23.340000,23.440001,22.832674,9000
...,...,...,...,...,...,...,...
246,2023-07-12,29.770000,30.011000,29.770000,30.000000,30.000000,6400
247,2023-07-13,30.389999,30.459999,30.350000,30.459999,30.459999,4900
248,2023-07-14,30.309999,30.309999,30.309999,30.309999,30.309999,0
249,2023-07-17,30.170000,30.430000,30.120001,30.430000,30.430000,2900


## Datenbanken

Das Speichern und Abrufen mittels csv, txt oder ähnlicher Dateiformate ist in manchen Fällen nicht ausreichend. Beispielsweise erreichen csv oder Excel Dateien ihr Limit bei ca. einer Million Beobachtungen. Dies mag viel erscheinen, ist es jedoch in manchen Fällen nicht. Zudem ist es oft besser Daten in etwas komplexerer Form strukturieren zu können. Hierfür bieten sich Datenbanken an, über die Daten mittels einer "query language" abgerufen werden können. Eine der für diesen Zweck bekanntesten Sprachen (für relationelle Datenbanken) ist Structured Query Language - SQL. Hiermit können spezifische Werte durch einzelne Befehle aus der Datenbank abgefragt werden. Um Ihnen ein kleines Beispiel zu geben, sehen Sie in den nächsten drei Zellen, wie Daten in eine Datenbank geschrieben und wieder abgefragt werden können.

In [4]:
import sqlite3
import pandas as pd

# Aufbau einer Verbindung zur Datenbank dax.sqlite
conn = sqlite3.connect("dax.sqlite")
# Schreiben von Daten mit Hilfe eines pandas DataFrame
dax_df.to_sql(con = conn, name = "ohlc", if_exists="replace")
conn.close()

In [20]:
import sqlite3

# Beispiel für den Abruf aller Daten aus dem Tabellenblatt ohlc
conn = sqlite3.connect("dax.sqlite")
curs = conn.cursor()
# dies ist der SQL Befehl
sql_query = "SELECT * FROM ohlc"
res = curs.execute(sql_query)
for i, line in enumerate(res):
    print(line)
    if i == 5:
        break
conn.close()

(0, '2022-07-19', 23.18, 23.719999, 23.17, 23.59, 22.978786, 74900)
(1, '2022-07-20', 23.43, 23.6, 23.280001, 23.35, 22.745005, 17100)
(2, '2022-07-21', 23.15, 23.51, 23.15, 23.51, 22.90086, 4900)
(3, '2022-07-22', 23.74, 23.74, 23.33, 23.33, 22.725523, 4700)
(4, '2022-07-25', 23.6, 23.6, 23.34, 23.440001, 22.832674, 9000)
(5, '2022-07-26', 23.040001, 23.139999, 22.93, 22.965, 22.36998, 3500)


In [23]:
import sqlite3

# Beispiel für den Abruf aller Daten ab einem bestimmten Daten aus dem Tabellenblatt ohlc
conn = sqlite3.connect("dax.sqlite")
curs = conn.cursor()
sql_query = "SELECT * FROM ohlc WHERE Date>='2022-08-01'"
res = curs.execute(sql_query)
for i, line in enumerate(res):
    print(line)
    if i == 5:
        break
conn.close()

(9, '2022-08-01', 24.1, 24.209999, 23.771, 24.004, 23.382059, 44200)
(10, '2022-08-02', 23.82, 23.92, 23.639999, 23.639999, 23.027491, 18600)
(11, '2022-08-03', 23.940001, 24.08, 23.700001, 24.040001, 23.417128, 21600)
(12, '2022-08-04', 24.215, 24.32, 24.195, 24.23, 23.602203, 11200)
(13, '2022-08-05', 24.0, 24.15, 23.940001, 24.120001, 23.495054, 4200)
(14, '2022-08-08', 24.24, 24.35, 24.052, 24.156, 23.530123, 8000)


## APIs

Oft teilen Firmen Daten auf ihren Servern mittels eines Application Programming Interfaces (API). Sie können sich dies wie eine Speisekarte in einem Restaurant vorstellen. Der Anbieter (das Restaurant) hat Daten auf seinem Server (die Gerichte in der Küche). Um Essen bestellen zu können, benötigen Sie die Informationen zu den verschiedenen Gerichten. Dies wären die Endpunkte des APIs. Wenn Sie diese kennen, sind Sie in der Lage eine Ihren Wünschen entsprechende Bestellung aufzugeben und bekommen im besten Fall unmittelbar die angefragten Daten. 

Als kleines Beispiel benützen wir die Star Wars API. Folgende Endpunkte sind beispielsweise verfügbar:

* https://swapi.dev/api/films
* https://swapi.dev/api/people 
* https://swapi.dev/api/planets
* https://swapi.dev/api/species
* https://swapi.dev/api/starships
* https://swapi.dev/api/vehicles

Wenn Sie nun Informationen zu Personen aus dem Star Wars Universum benötigen, können Sie den Endpunkt https://swapi.dev/api/people verwenden. Mittels Python senden Sie eine GET Anfrage, um die Daten zu bestellen. Sie erhalten die entsprchenden Daten im json Format zurück, welches für diese Art der Datenabfrage sehr beliebt ist. Durch pandas lassen sich die Daten schnell und einfach in einen leicht zu analysierenden Datensatz konvertieren.

In [4]:
import requests
import pandas as pd

# Senden der GET Anfrage
r = requests.get("https://swapi.dev/api/people/")
# So sehen die übermittelten Daten im json-Format aus
# In Python werden diese als dictionary dargestellt
r.json()

{'count': 82,
 'next': 'https://swapi.dev/api/people/?page=2',
 'previous': None,
 'results': [{'name': 'Luke Skywalker',
   'height': '172',
   'mass': '77',
   'hair_color': 'blond',
   'skin_color': 'fair',
   'eye_color': 'blue',
   'birth_year': '19BBY',
   'gender': 'male',
   'homeworld': 'https://swapi.dev/api/planets/1/',
   'films': ['https://swapi.dev/api/films/1/',
    'https://swapi.dev/api/films/2/',
    'https://swapi.dev/api/films/3/',
    'https://swapi.dev/api/films/6/'],
   'species': [],
   'vehicles': ['https://swapi.dev/api/vehicles/14/',
    'https://swapi.dev/api/vehicles/30/'],
   'starships': ['https://swapi.dev/api/starships/12/',
    'https://swapi.dev/api/starships/22/'],
   'created': '2014-12-09T13:50:51.644000Z',
   'edited': '2014-12-20T21:17:56.891000Z',
   'url': 'https://swapi.dev/api/people/1/'},
  {'name': 'C-3PO',
   'height': '167',
   'mass': '75',
   'hair_color': 'n/a',
   'skin_color': 'gold',
   'eye_color': 'yellow',
   'birth_year': '112BB

In [32]:
# mittels pandas lassen sich die json Daten in eine DataFrame konvertieren
star_wars_pepole_df = pd.DataFrame(r.json()['results'])
star_wars_pepole_df.head()

,name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,homeworld,films,species,vehicles,starships,created,edited,url
0,Luke Skywalker,172,77,blond,fair,blue,19BBY,male,https://swapi.dev/api/planets/1/,"[https://swapi.dev/api/films/1/, https://swapi...",[],"[https://swapi.dev/api/vehicles/14/, https://s...","[https://swapi.dev/api/starships/12/, https://...",2014-12-09T13:50:51.644000Z,2014-12-20T21:17:56.891000Z,https://swapi.dev/api/people/1/
1,C-3PO,167,75,n/a,gold,yellow,112BBY,n/a,https://swapi.dev/api/planets/1/,"[https://swapi.dev/api/films/1/, https://swapi...",[https://swapi.dev/api/species/2/],[],[],2014-12-10T15:10:51.357000Z,2014-12-20T21:17:50.309000Z,https://swapi.dev/api/people/2/
2,R2-D2,96,32,n/a,"white, blue",red,33BBY,n/a,https://swapi.dev/api/planets/8/,"[https://swapi.dev/api/films/1/, https://swapi...",[https://swapi.dev/api/species/2/],[],[],2014-12-10T15:11:50.376000Z,2014-12-20T21:17:50.311000Z,https://swapi.dev/api/people/3/
3,Darth Vader,202,136,none,white,yellow,41.9BBY,male,https://swapi.dev/api/planets/1/,"[https://swapi.dev/api/films/1/, https://swapi...",[],[],[https://swapi.dev/api/starships/13/],2014-12-10T15:18:20.704000Z,2014-12-20T21:17:50.313000Z,https://swapi.dev/api/people/4/
4,Leia Organa,150,49,brown,light,brown,19BBY,female,https://swapi.dev/api/planets/2/,"[https://swapi.dev/api/films/1/, https://swapi...",[],[https://swapi.dev/api/vehicles/30/],[],2014-12-10T15:20:09.791000Z,2014-12-20T21:17:50.315000Z,https://swapi.dev/api/people/5/
